In [17]:
# Importing libraries
import pandas as pd

In [18]:
# Loading dataset
df = pd.read_csv("../data/processed/clean_retail_data.csv")
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [19]:
# Converting to datetime
df["invoicedate"] = pd.to_datetime(df["invoicedate"], errors="coerce")

# Dropping rows that failed conversion
df = df.dropna(subset=["invoicedate"])

df.dtypes
customer_df[["first_purchase", "last_purchase"]].dtypes

first_purchase    datetime64[ns]
last_purchase     datetime64[ns]
dtype: object

In [20]:
# Creating customer level dataframe from the dataset
customer_df = (
    df.groupby("customerid").agg(
        first_purchase = ("invoicedate", "min"),
        last_purchase = ("invoicedate", "max"),
        total_orders = ("invoiceno", "nunique"),
        total_revenue = ("revenue", "sum")
    )
    .reset_index()
)

customer_df.head()

,customerid,first_purchase,last_purchase,total_orders,total_revenue
0,12346.0,2011-01-18 10:01:00,2011-01-18 10:01:00,1,77183.60
1,12347.0,2010-12-07 14:57:00,2011-12-07 15:52:00,7,4310.00
2,12348.0,2010-12-16 19:09:00,2011-09-25 13:13:00,4,1797.24
3,12349.0,2011-11-21 09:51:00,2011-11-21 09:51:00,1,1757.55
4,12350.0,2011-02-02 16:01:00,2011-02-02 16:01:00,1,334.40


In [21]:
customer_df.describe()

,customerid,first_purchase,last_purchase,total_orders,total_revenue
count,4338.000000,4338,4338,4338.000000,4338.000000
mean,15300.408022,2011-04-30 17:06:50.857538048,2011-09-08 11:38:59.045643008,4.272015,2054.266460
min,12346.000000,2010-12-01 08:26:00,2010-12-01 09:53:00,1.000000,3.750000
25%,13813.250000,2011-01-17 11:13:15,2011-07-20 19:18:00,1.000000,307.415000
50%,15299.500000,2011-04-05 09:52:30,2011-10-20 10:40:30,2.000000,674.485000
75%,16778.750000,2011-08-19 10:11:30,2011-11-22 11:05:45,5.000000,1661.740000
max,18287.000000,2011-12-09 12:16:00,2011-12-09 12:50:00,209.000000,280206.020000
std,1721.808492,NaN,NaN,7.697998,8989.230441


In [22]:
customer_df["tenure_days"] = (
    customer_df["last_purchase"] - customer_df["first_purchase"]
).dt.days

In [23]:
customer_df.to_csv("../data/processed/customer_level_data.csv", index=False)